# Compare With Random Method


In [9]:
import os
import random
import numpy as np
import scipy.sparse

from exputils.dot.get_topK_Amat import get_topK_Amat
from exputils.dot.recovery_state import recovery_states_from_idxs
from exputils.extent.custom import calculate_extent_custom
from exputils.stabilizer_group import total_stabilizer_group_size
from exputils.state.random_ket import make_random_quantum_state
from exputils.extent.cg import calculate_extent_CG

In [10]:
def calculate_extent_random(n, psi, _K, _, rand_Amat):
    assert _K in np.linspace(0.1, 1.0, 10)
    Amat = scipy.sparse.hstack(
        [
            rand_Amat[:, : int(_K * rand_Amat.shape[1])],
            scipy.sparse.eye(2**n, format="csc", dtype=np.complex128),
        ]
    )
    assert Amat.shape[0] == (1 << n)
    assert Amat.shape[1] <= int(_K * total_stabilizer_group_size(n)) + 2**n
    print(f"random: {Amat.shape=}")
    return calculate_extent_custom(n, Amat, psi)[0]


def calculate_extent_dot(n, psi, _K, topK_Amat, _):
    assert _K in np.linspace(0.1, 1.0, 10)
    Amat = scipy.sparse.hstack(
        [
            topK_Amat[:, : int(_K * topK_Amat.shape[1])],
            scipy.sparse.eye(2**n, format="csc", dtype=np.complex128),
        ]
    )
    print(f"dot: {Amat.shape=}")
    return calculate_extent_custom(n, Amat, psi)[0]


def execute_experiments(n, maxK, kind):
    log_file_name = f"../data/random_vs_overlap/{kind}_{n}_data.txt"

    seed = 0
    psi = make_random_quantum_state(kind, n, seed)
    if os.path.exists(f"../data/CG/{kind}_{n}_data.txt"):
        with open(f"../data/CG/{kind}_{n}_data.txt") as f:
            exact = float(f.readlines()[-1].split()[0])
    else:
        exact = calculate_extent_CG(n, psi)[0]
    print(f"{exact=}")
    with open(log_file_name, mode="w") as f:
        print(f"{exact=}", file=f)

    t_s_g_s = total_stabilizer_group_size(n)
    topK_Amat = get_topK_Amat(n, psi, False, int(t_s_g_s * maxK), verbose=True)
    topK_Amat = topK_Amat[:, np.argsort(np.abs(topK_Amat.conj().T @ psi))[::-1]]
    rand_Amat = recovery_states_from_idxs(
        n, random.sample(range(t_s_g_s), int(t_s_g_s * maxK))
    )
    assert topK_Amat.shape[1] >= rand_Amat.shape[1], (topK_Amat.shape, rand_Amat.shape)
    topK_Amat = topK_Amat[:, : rand_Amat.shape[1]]

    methods = ["Random", "Overlap", "Exact"]
    functions = [calculate_extent_random, calculate_extent_dot, lambda *args: exact]
    for method, func in zip(methods, functions):
        for _K in np.linspace(0.1, 1.0, 10):
            if method != "Exact":
                print(f"{method=} {_K=:.3e}", end=" ")
            extent = func(n, psi, _K, topK_Amat, rand_Amat)
            K = _K * maxK
            with open(log_file_name, mode="a") as f:
                print(f"{n=}, {seed=}, {K=:.3e}, {method=}, {extent=}", file=f)
    print("done")


if __name__ == "__main__":
    # execute_experiments(5, 1e-4, "pure")
    execute_experiments(7, 1e-7, "pure")
    execute_experiments(8, 1e-10, "pure")
    execute_experiments(9, 1e-12, "pure")

exact=7.380465055062293
[k|progress|range]: 1 | 3.26400e+04/8.12849e+10 | [0.10391, 0.30373] | 31[ms]
[k|progress|range]: 2 | 2.76365e+06/8.12849e+10 | [0.21379, 0.34968] | 205[ms]
[k|progress|range]: 3 | 9.95194e+07/8.12849e+10 | [0.26923, 0.37190] | 568[ms]
[k|progress|range]: 4 | 1.64761e+09/8.12849e+10 | [0.30431, 0.41110] | 902[ms]
[k|progress|range]: 5 | 1.28338e+10/8.12849e+10 | [0.32744, 0.41110] | 1026[ms]
[k|progress|range]: 6 | 4.69251e+10/8.12849e+10 | [0.34131, 0.41908] | 1171[ms]
[k|progress|range]: 7 | 8.12849e+10/8.12849e+10 | [0.34669, 0.41908] | 1291[ms]
 calculation time : 1291[ms]
method='Random' _K=1.000e-01 random: Amat.shape=(128, 940)
method='Random' _K=2.000e-01 random: Amat.shape=(128, 1753)
method='Random' _K=3.000e-01 random: Amat.shape=(128, 2566)
method='Random' _K=4.000e-01 random: Amat.shape=(128, 3379)
method='Random' _K=5.000e-01 random: Amat.shape=(128, 4192)
method='Random' _K=6.000e-01 random: Amat.shape=(128, 5004)
method='Random' _K=7.000e-01 rand